This notebook allows the reproduce **Table 3** of the paper "Covering the Combinatorial Design Space of Multiplex CRISPR/Cas Experiments in Plants" by Kirsten Van Huffel, Michiel Stock, Thomas Jacobs, Tom Ruttink and Bernard De Baets.

#### Load packages

In [3]:
using Random 
using Plots  
using Distributions 
using LinearAlgebra
using Combinatorics
using BioCCP


(process:12136): GLib-GIO-WARNING **: 15:54:13.384: Unexpectedly, UWP app `HaukeGtze.7-ZipFileManagerUnofficial_1.2107.2.0_x64__6bk20wvc8rfx2' (AUMId `HaukeGtze.7-ZipFileManagerUnofficial_6bk20wvc8rfx2!zFM.exe') supports 36 extensions but has no verbs


#### Load BioCCP- and simulation-based framework

In [8]:
include("MultiplexCrisprDOE.jl");

#### Set seed for reproducibility

In [ ]:
Random.seed!(1)

#### Calculate expected plant library size for full coverage and measure execution time for both approaches

1) CRISPR/Cas experiment targeting single gene knockouts

In [11]:
include("DefaultParameters_k=1.jl");

In [12]:
@time simulate_Nₓ₁(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global; iter=500)

  0.422180 seconds (1.86 M allocations: 236.583 MiB, 11.03% gc time, 36.99% compilation time)


(107.79, 35.02220738360221)

In [13]:
@time BioCCP_Nₓ₁(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global)

  0.058961 seconds (131.27 k allocations: 6.807 MiB, 97.34% compilation time)


(110, 38)

2) Multiplex CRISPR/Cas experiment targeting pairwise combinations of gene knockouts

In [14]:
include("DefaultParameters_k=2.jl");

In [15]:
@time simulate_Nₓ₂(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global; iter=500)

  6.592268 seconds (21.82 M allocations: 6.924 GiB, 12.65% gc time)


(2785.944, 653.5442303210796)

In [16]:
@time BioCCP_Nₓ₂(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global)

  0.761519 seconds (1.72 M allocations: 113.445 MiB, 2.48% gc time, 97.28% compilation time)


(2731, 641)

3) Multiplex CRISPR/Cas experiment targeting triple combinations of gene knockouts

In [18]:
include("DefaultParameters_k=3.jl");

In [ ]:
@time simulate_Nₓ₃(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global; iter=500)

In [13]:
n_gRNA_perconstruct = 3
Random.seed!(1)
@time BioCCP_Nₓ₃(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global)

  3.184293 seconds (69.43 M allocations: 1.176 GiB, 5.36% gc time, 27.13% compilation time)


(36771, 6989)